In [220]:
# Importing packages

import pandas as pd
import numpy as np
import plotnine as p9

In [221]:
# Setting routes for reading in csv data

data_masterplan_input_target = 'test_mp_data.csv'
phasing_logic_input_target = 'phasing_logic_matrix.csv'

# Reading in masterplan data and phasing logic matrix

data_masterplan_input =  pd.read_csv(data_masterplan_input_target)
phasing_logic_input = pd.read_csv(phasing_logic_input_target).rename({'delivery_status':'Phasing_Note'},axis = 1)


In [222]:
# Dropnas from the project ID column from the main data_masterplan_input data frame

data_masterplan_input = data_masterplan_input.dropna(subset = ['ProjectID'],axis = 0).reset_index(drop = True)

In [223]:
# Find columns with a '%' sign and the column immediately to the left in the main masterplan input dataframe

columns_with_percent = [col for col in data_masterplan_input.columns if '%' in col]

selected_columns = []

for col in columns_with_percent:

    # Find the index of the column with a '%' sign
    
    col_index = data_masterplan_input.columns.get_loc(col)
    # Append the column to the left if it exists
    
    if col_index > 0:
        selected_columns.append(data_masterplan_input.columns[col_index - 1])
    
    # Append the column with a '%' sign
    
    selected_columns.append(col)

# Select the identified columns from the DataFrame
committed_development = data_masterplan_input[selected_columns]
committed_development = pd.concat([data_masterplan_input[['ProjectID','Phasing_Note','Approved Completion Year']],committed_development],axis = 1)

In [224]:
# Creating dataframe that contains land uses for joining onto adhoc dataframes later in the phasing process.

land_use_df = pd.DataFrame({'land_use':selected_columns})
land_use_df = land_use_df[~land_use_df['land_use'].str.contains('%')].reset_index(drop = True)

In [225]:
# Setting start dates of the plan

plan_start = 2024
plan_end = 2040

# Creating sequence containing all years of the plan phasing period

phasing_period = np.arange(plan_start,plan_end+1,1)
phasing_period = phasing_period.astype(str)
print(phasing_period)


['2024' '2025' '2026' '2027' '2028' '2029' '2030' '2031' '2032' '2033'
 '2034' '2035' '2036' '2037' '2038' '2039' '2040']


In [226]:
# Creating dataframe includes the manual phasing overrides

phasing_override_input = data_masterplan_input[phasing_period]
phasing_override_input = pd.concat([data_masterplan_input['ProjectID'],phasing_override_input],axis = 1)

# Creating data frame with phasing timeline for attaching to different development phasing pipelines

timeline = phasing_override_input

In [227]:
# Identifying which development group committed development should be entered in based on delivery status

committed_development =committed_development.merge(phasing_logic_input,how = 'left',on = 'Phasing_Note')

# Dropping masterplans without a status or are complete

committed_development= committed_development.dropna(subset=['group'],axis =0)
committed_development = committed_development[committed_development['group'] != 'Remove'].reset_index(drop = True)


In [251]:
# Creating dataframe containing total development available in each masterplan area

total_development = committed_development.drop(['Phasing_Note','Approved Completion Year','group','delay_period_yrs'],axis = 1)
total_development = total_development.melt(id_vars= 'ProjectID').reset_index(drop = True) 
total_development= total_development[~total_development['variable'].str.contains('%')].reset_index(drop = True)
total_development = total_development.rename({'variable':'land_use'},axis =1 )

In [228]:
# Creating a data frame that contains under construction development
under_construction_development = committed_development[committed_development['group'] == 'Under Construction'].reset_index(drop = True)
under_construction_development['Approved Completion Year'] =under_construction_development['Approved Completion Year'].apply(lambda x: 1 if pd.isna(x) or not isinstance(x, int) else x
).astype(int)

# Removing un-needed columns
under_construction_development = under_construction_development.drop(['group','delay_period_yrs','Phasing_Note'],axis = 1)

# Melting the data frame into long form
under_construction_development = under_construction_development.melt(id_vars= ['ProjectID','Approved Completion Year'])

# Splitting the long-form data frame into two parts and merging those parts together - the purpose being to split the % committed and number of units associated with each land use into separatre columns
under_construction_development_split_1 = under_construction_development[~under_construction_development['variable'].str.contains('%')].reset_index(drop = True).rename({'variable':'land_use','value':'number_units'},axis = 1)
under_construction_development_split_2 = under_construction_development[under_construction_development['variable'].str.contains('%')].reset_index(drop = True).drop(['ProjectID','variable','Approved Completion Year'],axis = 1).rename({'value':'per_committed'},axis = 1)
under_construction_development = pd.concat([under_construction_development_split_1,under_construction_development_split_2],axis = 1)

# Creating starting and end of phasing year columns

under_construction_development['start_year'] = plan_start
under_construction_development['end_year'] = under_construction_development['Approved Completion Year'].apply(lambda x: (plan_start + 1) if pd.isna(x) or x <= plan_start else x)

# Calculating the percentage of development over the course of each phasing period

under_construction_development['development_pa'] = under_construction_development['per_committed']/(under_construction_development['end_year'] - under_construction_development['start_year'])



In [229]:
# Merging data and preprocessing
under_construction_development_phased = under_construction_development.merge(timeline, how='left', on='ProjectID')
under_construction_development_phased = under_construction_development_phased.drop(['Approved Completion Year', 'number_units', 'per_committed'], axis=1)
under_construction_development_phased = under_construction_development_phased.melt(id_vars=['ProjectID', 'land_use', 'start_year', 'end_year', 'development_pa'])

# Ensuring that datatype conversions are handled for all related columns
under_construction_development_phased['variable'] = pd.to_numeric(under_construction_development_phased['variable'], errors='coerce')  # Converting the 'variable' which probably holds year values
under_construction_development_phased['start_year'] = pd.to_numeric(under_construction_development_phased['start_year'], errors='coerce')  # Converting 'start_year' to numeric if it isn't already
under_construction_development_phased['end_year'] = pd.to_numeric(under_construction_development_phased['end_year'], errors='coerce')  # Converting 'end_year' to numeric
under_construction_development_phased['development_pa'] = pd.to_numeric(under_construction_development_phased['development_pa'], errors='coerce')  # Converting 'development_pa'

# Handling the conditional logic to determine the value based on year range
under_construction_development_phased['value'] = under_construction_development_phased.apply(
    lambda row: row['development_pa'] if (row['variable'] >= row['start_year'] and row['variable'] <= row['end_year']) else 0, 
    axis=1)

under_construction_development_phased = under_construction_development_phased[['ProjectID','land_use','variable','value']].rename({'variable':'year','value':'development_pa'},axis =1)
under_construction_development_phased['status'] = 'Under Construction'

In [230]:
# Creating a data frame that contains under construction development
pipeline_development = committed_development[committed_development['group'] == 'Pipeline'].reset_index(drop = True)
pipeline_development['Approved Completion Year'] = pipeline_development['Approved Completion Year'].apply(lambda x: 1 if pd.isna(x) or not isinstance(x, int) else x
).astype(int)

# Removing un-needed columns
pipeline_development= pipeline_development.drop(['group','Phasing_Note'],axis = 1)

# Melting the data frame into long form
pipeline_development= pipeline_development.melt(id_vars= ['ProjectID','Approved Completion Year','delay_period_yrs'])

# Splitting the long-form data frame into two parts and merging those parts together - the purpose being to split the % committed and number of units associated with each land use into separatre columns
pipeline_development_split_1 = pipeline_development[~pipeline_development['variable'].str.contains('%')].reset_index(drop = True).rename({'variable':'land_use','value':'number_units'},axis = 1)
pipeline_development_split_2 = pipeline_development[pipeline_development['variable'].str.contains('%')].reset_index(drop = True).drop(['ProjectID','variable','Approved Completion Year','delay_period_yrs'],axis = 1).rename({'value':'per_committed'},axis = 1)
pipeline_development = pd.concat([pipeline_development_split_1,pipeline_development_split_2],axis = 1)
pipeline_development['delay_period_yrs'] = pipeline_development['delay_period_yrs'].astype(int)

# Creating starting and end of phasing year columns

pipeline_development['start_year'] =  pipeline_development['delay_period_yrs'] + plan_start
pipeline_development['end_year'] = pipeline_development['Approved Completion Year'].apply(lambda x: plan_end if pd.isna(x) or x <= plan_start or x < plan_end else x)

# Calculating the percentage of development over the course of each phasing period
pipeline_development['development_pa'] = pipeline_development['per_committed']/(pipeline_development['end_year'] - pipeline_development['start_year'])


In [231]:
# Merging data and preprocessing
pipeline_development_phased = pipeline_development.merge(timeline, how='left', on='ProjectID')
pipeline_development_phased = pipeline_development_phased.drop(['Approved Completion Year', 'number_units', 'per_committed','delay_period_yrs'], axis=1)
pipeline_development_phased = pipeline_development_phased.melt(id_vars=['ProjectID', 'land_use', 'start_year', 'end_year', 'development_pa'])

# Ensuring that datatype conversions are handled for all related columns
pipeline_development_phased['variable'] = pd.to_numeric(pipeline_development_phased['variable'], errors='coerce')  # Converting the 'variable' which probably holds year values
pipeline_development_phased['start_year'] = pd.to_numeric(pipeline_development_phased['start_year'], errors='coerce')  # Converting 'start_year' to numeric if it isn't already
pipeline_development_phased['end_year'] = pd.to_numeric(pipeline_development_phased['end_year'], errors='coerce')  # Converting 'end_year' to numeric
pipeline_development_phased['development_pa'] = pd.to_numeric(pipeline_development_phased['development_pa'], errors='coerce')  # Converting 'development_pa'

# Handling the conditional logic to determine the value based on year range
pipeline_development_phased['value'] = pipeline_development_phased.apply(
    lambda row: row['development_pa'] if (row['variable'] >= row['start_year'] and row['variable'] <= row['end_year']) else 0, 
    axis=1)

pipeline_development_phased = pipeline_development_phased[['ProjectID','land_use','variable','value']].rename({'variable':'year','value':'development_pa'},axis =1)
pipeline_development_phased['status'] = 'Pipeline'

In [256]:
pipeline_development_phased

,ProjectID,land_use,year,development_pa,status
0,Samha New Housing - Ph 3,Residential - Villas Emiratis (Units),2024,0.000000,Pipeline
1,Samha New Housing - Ph 2,Residential - Villas Emiratis (Units),2024,0.000000,Pipeline
2,ABU MREIKHAH_2,Residential - Villas Emiratis (Units),2024,0.000000,Pipeline
3,26626.0,Residential - Villas Emiratis (Units),2024,0.000000,Pipeline
4,Umm Al Nar,Residential - Villas Emiratis (Units),2024,0.000000,Pipeline
...,...,...,...,...,...
28351,10234.0,Other (GFA),2040,0.000000,Pipeline
28352,AL SMEEH_21,Other (GFA),2040,0.000000,Pipeline
28353,GHANADHAH_2,Other (GFA),2040,0.000000,Pipeline
28354,1113.13,Other (GFA),2040,0.090909,Pipeline


In [232]:
# Collating under construction and pipeline development into one section

development_phased = pd.concat([under_construction_development_phased,pipeline_development_phased],axis = 0).reset_index(drop = True)

In [233]:
# Creating data frame containing the phased development

override_phased = phasing_override_input.melt(id_vars= 'ProjectID').reset_index(drop = True)
override_phased['value'] = override_phased.apply(lambda row: 0.05 if row['ProjectID'] == 'Riyadh' else row['value'],axis = 1)
override_phased = override_phased.rename({'variable':'year','value':'override_development_pa'},axis =1).dropna().reset_index(drop = True)

# Picking out developments that have associated phasings

override_mps = override_phased['ProjectID'].drop_duplicates()
override_mps = override_mps.tolist()

In [234]:
# Creating function for creating a sub-dataframe that contains the project ids and land uses in long form

def land_use_df_concat(land_use_df, land_uses):
    # Create a copy of the original DataFrame to avoid modifying it directly
    dummy_df = land_use_df.copy()
    dummy_df['ProjectID'] = 'a_number'
    
    # Initialize a list to store DataFrames
    df_list = [dummy_df]
    
    for use in land_uses:
        new_df = land_use_df.copy()
        new_df['ProjectID'] = use
        df_list.append(new_df)
    
    # Concatenate all DataFrames in the list along the rows (axis=0)
    result_df = pd.concat(df_list, axis=0)
    
    return result_df

In [235]:
# Using land use sub-dataframe and binding on phasing to create a phasing by land use timeline for each project

override_phased = override_phased.pivot(index = ['ProjectID'], columns = ['year'],values = 'override_development_pa').reset_index()
override_land_use_df = land_use_df_concat(land_use_df,override_mps)
override_land_use_df = override_land_use_df[override_land_use_df['ProjectID'] != 'a_number']
override_phased = override_phased.merge(override_land_use_df,how = 'left', on = 'ProjectID')

# Shifting override phased dataframe to long form for merging with the phased development dataframe

override_phased = override_phased.melt(id_vars = ['ProjectID','land_use'])
override_phased = override_phased.rename({'variable':'year','value':'development_pa'},axis = 1)
override_phased['status'] = 'Override'

In [236]:
# Filtering out masterplans that have been have been overridden

development_phased = development_phased[~development_phased['ProjectID'].isin(override_mps)]

# Concatenating the phased override dataframe with the filtered phased development dataframe

development_phased = pd.concat([development_phased,override_phased],axis = 0)

In [267]:
test= development_phased.merge(total_development,how = 'left', on = ['ProjectID','land_use'])
test = test.rename({'development_pa':'percent_development_pa','value':'development_pa'},axis = 1)
test = pd.to_numeric(test['development_pa'], errors='coerce').fillna(0)
test['development_pa'] = test['development_pa'] * test['percent_development_pa']
test

KeyError: 'development_pa'

In [257]:
(p9.ggplot(development_phased,p9.aes(x = year,
                                     y = 'd')))

,ProjectID,land_use,year,development_pa,status
1,1099.0,Residential - Villas Emiratis (Units),2024,0.00,Under Construction
2,2100.0,Residential - Villas Emiratis (Units),2024,1.00,Under Construction
3,1170.1,Residential - Villas Emiratis (Units),2024,1.00,Under Construction
4,5000.0,Residential - Villas Emiratis (Units),2024,1.00,Under Construction
5,1996.0,Residential - Villas Emiratis (Units),2024,1.00,Under Construction
...,...,...,...,...,...
199,Riyadh,Hotel (Units),2040,0.05,Override
200,Riyadh,Community Facilities (GFA),2040,0.05,Override
201,Riyadh,Healthcare (GFA),2040,0.05,Override
202,Riyadh,Education (GFA),2040,0.05,Override
